In [1]:
import requests
from bs4 import BeautifulSoup
import json 
import pymysql
import time

In [2]:
def vivaUrl(search):
    url_temp = 'http://www.viva100.com/main/search_detail.php?&schLCode=&schMCode=&schSCode=&schBoardCode=&schBoardCode1=&s_word={search}&w_gbn=all&startday=2019-04-01&endday=2019-09-30'.format(search=search)
    for pageNo in range(1, 25):
        url = url_temp  + "&page={pageNo}".format(pageNo=pageNo)
        html = requests.get(url)  
        bs = BeautifulSoup(html.content,"html.parser")
        news_urls = []
        u = "http://www.viva100.com/main/"
        for url_no in range(len(bs.select(".allnews_list .allnews_tit"))):
            news_urls.append(u+ bs.select(".allnews_list .allnews_tit")[url_no].select("a")[0].get("href"))
        vivaInfo(news_urls)

In [6]:
def vivaInfo(news_urls):
    for url_no in range(len(news_urls)):
        url2 = news_urls[url_no]
        html2 = requests.get(url2)
        bs2 = BeautifulSoup(html2.content,"html.parser")
#         time.sleep(2)
        try:
            # 제목
            title = bs2.select(".view_top_title")[0].text
            
            # 부제목
            if bs2.select(".view_top_text") == []:
                sub_title=""
            else:
                sub_title = bs2.select(".view_top_text")[0].text
        
            # 날짜
            newsDate = bs2.select(".view_top_days.view1_top_days")[0].text
            newsDate = newsDate.replace("입력 ", "").replace("-", "")[0:8]
            
            # 본문
            content = []
            for content_no in range(len(bs2.select(".left_text_box"))):
                content.append(bs2.select(".left_text_box")[content_no].text)
            content_replace = content[0].replace("\n","").replace("\t","").replace("\r","")

            print(url2)
            print("제목 : ",title)
            print("부제목 : ",sub_title)
            print(newsDate)
            print("본문 :",content_replace)
            print("============================================================")
        except:
            print("<<<<<에러 url:>>>>>>", url2)
            continue
            
        dbData = [[url2, title, newsDate, sub_title, content_replace]]
        connectDB(dbData)

In [4]:
def connectDB(dbData):
    DB_HOST = '127.0.0.1'
    DB_USER = 'root'
    DB_PASSWD = 'autoset'
    DB_NAME = 'final'
    
    conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                          db=DB_NAME, charset='utf8')
    curs = conn.cursor()
    sql = """insert into viva100_bs(url, title, date, sub_title, contents) values (%s, %s, %s, %s, %s)"""
    curs.executemany(sql,dbData)
    conn.commit()
    
    conn.close()

In [8]:
search = '일본 여행'
# 보통 우리가 무엇을 검색해서 크롤링 할 지 정해져있기 때문에 미리 저장해둠.
vivaUrl(search)

http://www.viva100.com/main/view.php?key=20191004010001183
제목 :  유안타증권 “여행약세 장기화…하나·모두투어 목표주가 하향”
부제목 :  
20191004
본문 : 유안타증권은 4일 “여행 업황 약세 국면이 장기화될 가능성이 높다”며 하나투어와 모두투어의 투자의견을 각각 ‘보유’로 하향 조정하고, 목표주가도 내렸다.유안타증권 박성호 연구원은 “원화가치 하락과 내수경기 불안이 해외여행 수요약세 요인으로 작용하는 가운데, 일본상품 불매운동으로 인해 일본 여행수요가 급락하면서 여행 업황은 지난 2011년 일본 대지진 당시보다도 안 좋아졌다”고 진단했다.박 연구원은 “하나투어 기준으로 보면, 2008년 금융위기 직후보다 조금 나은 수준”이라며 “하나투어와 모두투어의 12월 예약률 감소폭이 10~11월보단 완화되는 모습을 보이고 있지만, 아직 바닥권 통과 신호로 보기는 이르다”고 분석했다.그는 “한일갈등이 장기화될 경우, 일부 일본 여행수요가 중국, 동남아 등으로 대체될 수 있겠지만 전체 수요 증가를 의미있는 수준으로 견인하기엔 역부족일 것”이라고 예상했다.박 연구원은 “하나투어, 모두투어의 주가는 상당히 싸진 상태지만, 여행업황 약세 국면이 장기화될 가능성이 있어 바닥권 매집 시점으로는 이른 시기라고 판단한다”며 “하나투어와 모두투어에 대한 투자의견을 ‘매수’에서 ‘보유’로 하향 조정하고, 목표주가도 4만4000원, 1만6000원으로 내린다”고 밝혔다.이어 “현재의 업황국면에선 일본 비중이 상대적으로 적은 모두투어에 대한 선호도가 그나마 좀 높다”며 “한일 갈등 해소 및 여행 업황 강세 전환 시, 반대로 하나투어의 주가 상승탄력이 상대적으로 클 것”이라고 덧붙였다.이은혜 기자 chesed71@viva100.com기자의 다른기사보기 >
http://www.viva100.com/main/view.php?key=20191002010000715
제목 :  카카오, 동남아 모빌리티 최강자 ‘그랩’ 손잡고 베트남 진출
부제목 :

http://www.viva100.com/main/view.php?key=20190926010008636
제목 :  에어서울, 12월 베트남 하노이 신규 취항
부제목 :  다낭·나트랑까지 베트남 3개 노선 보유
20190926
본문 : 에어서울 항공기 (사진제공=에어서울) 에어서울은 오는 12월16일부터 베트남 하노이 노선에 주 7회 일정으로 신규 취항한다고 26일 밝혔다.에어서울은 신규 취항을 맞아 이날 오후 3시부터 특가 프로모션을 실시한다. 편도총액운임은 5만2100원이며, 탑승기간은 첫 취항 날부터 2020년 3월28일까지다.에어서울 관계자는 “12월 하노이와 나트랑에 취항함으로써 기존의 다낭을 포함해 베트남 인기 도시 3개 노선을 운항하게 돼 여행객들은 더욱 자유롭고 다채로운 여행 루트를 계획할 수 있을 것”이라며 “일본 노선 비중을 계속 축소해 동남아 등 중거리 중심으로 노선을 다변화 하겠다”고 말했다.이효정 기자 hyo@viva100.com 기자의 다른기사보기 >
http://www.viva100.com/main/view.php?key=20190925010008148
제목 :  한·일 갈등에 지방공항 日노선 승객 34% ‘뚝’
부제목 :  윤관석 의원 “지방공항 日노선 편중 심해, 수입도 타격” 우려
20190925
본문 : 일본의 수출규제에 따른 불매운동으로 지방 공항의 탑승객이 3분의 1 가까이 줄었다. 특히, 개별 공항에 따라 수입이 최대 70% 급감한 것으로 드러났다.  25일 국회 국토교통위원회 소속 윤관석 의원이 한국공항공사로부터 받은 ‘7개 국제선공항 운항·여객·수입 현황’에 따르면 일본의 ‘백색국가 한국 제외’ 방침 발표 직후인 7월 첫 주 1029편이었던 일본 노선 운항편 수는 불과 두 달만에 836편으로 18.8% 축소됐다. 같은 기간 이들 공항의 일본 노선 여객 수도 15만1283명에서 5만1133명으로 33.8%나 줄었다. 한국공항공사가 운영·관리하는 7개 공항은 김포·김해·제주·대구·청주·무안·양양 공항이다. 비행기 운항과 

http://www.viva100.com/main/view.php?key=20190923010007439
제목 :  올 추석, 국내 여행 선호… 야놀자, 추석 연휴 여가 트렌드 발표
부제목 :  
20190923
본문 : 글로벌 여가 플랫폼 기업 야놀자가 올 추석 연휴 국내 및 해외 여가 트렌드를 분석해 23일 발표했다.이번 추석 연휴기간동안 실제 예약 데이터를 바탕으로 조사한 결과 국내 숙소 예약 건수는 지난해 추석 연휴기간보다 143% 증가했다. 이는 전년 대비 하루 짧은 연휴기간과 일본 여행 기피현상이 겹쳐 국내 관광 수요가 늘어난 결과로 풀이된다.지역별 숙소 예약률은 서울(24.9%)이 1위를 차지했다. 경기도(22.1%)가 근소한 차이로 2위를, 부산(9.2%)이 3위에 올랐다. 지난해에 이어 올해도 서울과 경기도가 인기 상위지역으로 꼽혀 연휴기간 상대적으로 한산한 수도권 지역의 선호도가 높은 것으로 분석된다.숙소 유형별로는 펜션(147%)이 전년 대비 가장 큰 증가세를 보이며 주류 여가 트렌드로 자리잡은 펜캉스의 인기를 입증했다. 이어 모텔(144%), 호텔(134%), 게스트하우스(92%) 등의 순이다. 특히 호텔은 가장 높은 당일 예약률(37%)을 보였다. 이는 국내 즉흥여행 트렌드와 호캉스족 증가로 보인다는 견해다. 국내 레저 예약 역시 전년 대비 400%의 높은 증가율을 기록했다. 유형별로는 스파 워터파크(40.6%), 테마파크(37.4%), 수상레저(9.1%) 순으로 예약률이 높았다. 특히 다양한 연령대가 함께 즐기기 좋은 스파 워터파크, 테마파크 등 대형시설이 전체의 78%를 차지했다. 연휴 기간 가족 단위로 즐길 수 있는 활동에 높은 선호도를 보인 것으로 나타났다.해외 숙소 예약률은 베트남(31.6%), 대만(16.3%), 태국(8.2%) 순이다. 전통적으로 인기 있는 동남아 국가들이 상위권을 차지했다. 도시 별로도 다낭(베트남), 타이베이(대만), 방콕(태국)이 각각 1~3위를 기록했다. 특히 대만은 지난달 발표한 미리 보는 연휴 트렌

http://www.viva100.com/main/view.php?key=20190918010006038
제목 :  ‘가지 않습니다’에 日여행업 직격탄…8월 방일 한국인 사상 최대폭 급감
부제목 :  
20190918
본문 : 지난 8월 8일 일본 불매운동 이후 직격탄을 맞아 한산한 대마도 히타카츠 항구 인근의 모습. (연합뉴스)‘가지 않습니다’ NO재팬 불매운동에 일본 여행산업이 직격탄을 맞았다.일본정부관광국(JNTO)이 18일 발표한 방일 외국인 여행자 통계(추계치)에 따르면 8월 일본을 찾은 한국인 여행자수는 30만8700명에 그쳐 전년 동월(59만3941명) 대비 -48%를 기록했다. 사상 최대폭 급감이다. 일본 정부의 경제 도발로 촉발된 일본 불매 운동이 시작된 첫 달(7월) 기록한 감소폭(-7.6%)의 6배에 가깝다.이로써 지난 1~8월 방일 한국인은 473만3100명으로 전년 동기 521만8264명과 비교해 -9.3%를 기록했다.한국인 감소의 영향으로 전월 일본을 찾은 전체 외국인수도 증가세가 꺾이면서, 전년 동기(257만8021명) 대비 2.2% 감소한 252만100명으로 집계됐다.일본 정부는 지난해에 이어 올해도 외국인 관광객 유치 3000만명을 목표로 잡고 있으나, 한국인 여행객 감소폭이 전체 증가세에 영향을 줄 정도가 되면서 이같은 목표에 차질이 생길 공산이 클 것으로 보인다.김수환 기자 ksh@viva100.com기자의 다른기사보기 >
http://www.viva100.com/main/view.php?key=20190918010005927
제목 :  bhc치킨, 일부 매장에 외국인 고객 ‘증가’
부제목 :  
20190918
본문 : bhc 종로점 전경 (사진제공=bhc)올해 들어 우리나라를 찾는 관광객이 늘어나고 있는 가운데 외국인 관광객들이 선호하는 음식으로 떠오르고 있는 치맥을 즐길 수 있는 매장이 덩달아 인기다. 서울시에 따르면 지난 1분기 서울을 찾은 외국인 관광객의 한국 음식에 대한 평가 결과 치킨이 불고기, 비빔밥에 이어 3번

http://www.viva100.com/main/view.php?key=20190916010004829
제목 :  인천공항, 추석연휴 총 여객 89만7078명, 일평균 17만9415명 이용
부제목 :  연휴기간 특별교통대책 실시로 전 분야 무결점 공항 운영 달성
20190916
본문 :                     인천국제공항공사는 올해 추석연휴 특별 교통대책 기간인 지난 11∼15일까지 약 89만7078 명, 일평균 기준 약 17만9415 명의 여객이 인천공항을 이용했다고 16일 밝혔다.   이는 지난해 추석연휴 기간인 일평균 18만7116 명 대비 4.1% 가량 감소한 수치로, 올해 연휴기간이 상대적으로 짧아 해외여행 수요가 감소하고 일본 노선 이용객이 전년대비 감소한 결과로 분석된다. 올해 추석연휴 기간 일평균 출발 여객은 8만7453 명으로 전년도 추석연휴 기간 10만950 명 대비 13.4% 가량 감소했했다.특히 일본 노선 일평균 출국여객은 전년대비 39.1% 가량 크게 줄었다.반면 올해 연휴 기간 일평균 도착 여객은 9만1962 명으로 전년도 추석 연휴 기간 8만6166 명 대비 6.7% 가량 증가했다. 이번 추석 연휴 기간 중 이용객이 가장 많았던 날은 지난 15일로 20만2223 명이 인천공항을 이용했으며, 출발여객이 가장 많은 날은 지난 12일(10만4903 명), 도착여객이 가장 많았던 날은 지난 15일 일요일(12만72 명)이었다. 터미널별로는 제1여객터미널 이용객 63만7957명(일평균 12만7591명), 제2여객터미널 이용객 25만 9121명(일평균  5만1824명)으로 여객이 고르게 분산(1터미널 71.1%, 2터미널 28.9%)됐다. 공사는 추석 연휴 기간 하루 평균 약 60여 명의 공사 특별근무인원과 370여 명의 안내요원을 현장에 배치하고 출국장 조기개장 및 연장운영, 기내 반입 금지물품 안내 강화, 공항철도 연장운행 및 심야버스 운행, 임시주차장 확보, 공항 주요시설 및 여객이용시설 안전점검 강화 등을 통해 무결점 공항

...

http://www.viva100.com/main/view.php?key=20190424010008952
제목 :  청주공항, 중국으로 가는 하늘길 넓어진다!
부제목 :  중국 웨이하이 정기노선 첫 취항, -신규 운수권 확보 마중물 역할 기대
20190424
본문 :                     청주국제공항에서 중국으로 가는 길이 넓어진다.24일 충북도는 중국 동방항공이 청주공항에서 처음으로 오는 5월 2일부터 국제 정기노선을 취항 한다고 밝혔다. 동항항공은 항공자유화(오픈스카이) 지역인 중국 산동성 웨이하이(위해)시와 청주공항을 잇는 웨이하이 정기노선을 A320(158석) 항공기로 주 3회(화,목,토) 운항할 계획이다. 한국과 가까운 도시이며, 중국 정부가 정한 ‘한중 협력 도시’이기도 한 웨이하이는 그동안 인천공항에서만 운항했으나 이번에 청주공항에서도 정기노선을 취항해 수도권 및 충청권 국민들의 항공 선택 기회가 확대될 뿐만 아니라 인천공항까지 가지 않아도 돼 편리하게 여행할 수 있게 됐다.이로 인해 청주국제공항의 중국 정기노선은 8개(북경, 항주, 상해, 연길, 대련, 심양, 닝보, 하얼빈)에서 9개 노선으로 확대 됐다.이밖에 도는 지난 3월 중순 한중 항공회담에서 양국 간 여객 운수권을 증대(주 548회→608회)키로 합의한 운수권 확대 계획에 따라 중국 노선 추가 확대를 위해 추진하고 있다. 도 관계자는 “중국으로 가는 하늘길을 추가로 확보하는 한편 일본, 베트남 등 아시아 지역의 다양한 국제노선 유치를 통해 청주공항의 경쟁력을 계속 강화해 나가겠다.”고 말했다. 충북=송태석 기자 0114662001@viva100.com기자의 다른기사보기 >
http://www.viva100.com/main/view.php?key=20190424010008599
제목 :  LF 바네사브루노, ‘카바스 보야지’ 글로벌 팝업스토어 국내 첫 오픈
부제목 :  
20190424
본문 : LF_바네사브루노 카바스 보야지 라움이스트 팝업스토어LF가 국내 전개하는 프랑스 

http://www.viva100.com/main/view.php?key=20190415010004852
제목 :  [비바100] 숨 막히는 해외여행… 안전한 고지대 여행을 위한 '고산병 예방법'
부제목 :  
20190416
본문 : (사진출처=게티이미지) 해외여행이 보편화 되면서 한국 사람들의 해외 여행지도 가까운 일본과 중국, 동남아를 넘어 유럽과 남미 지역으로 확대되고 있다. 특히 최근에는 북유럽의 알프스나 네팔 히말라야 등 유명한 산을 직접 등반하거나 고지대가 많은 남미 지역을 돌아보는 경우가 많다.해발이 낮은 산이나 지대는 별 문제가 없지만 해발이 높거나 고도가 높은 산은 조심해야 한다. 아무 생각없이 산에 올라갔다가 사망에 이를 수도 있기 때문이다. 바로 고산병이 문제다.고산병은 과거에는 산을 오르는 일부 등산객에게만 발생했지만 최근 해외 고도 높은 지역을 방문하는 여행객들이 증가하면서 발생률이 높아지고 있다. 고산병은 높은 산이나 고도에서 산소가 부족해져 생기는 저산소증이다. 개인차가 있지만 갑자기 높은 곳을 올라갔을 때 신체가 변화에 적응하지 못해 발생한다. 고산병이 발생하면 몸을 가누기 힘들어진다. 두통,  메스꺼움, 식욕부진, 피로, 어지럼증, 불면 등의 증상을 보이고, 심하면 뇌부종과 폐부종이 발생해 사망에 이르기도 한다. 전문 산악인이 아닌 일반인은 해발 2000~2500m에서는 약 22%, 해발 3000m에서는 약 42%가 고산병을 느낀다. 고지대는 대기 중 산소가 적어 혈액 속 조직에 저산소증이 발생한다. 이때 신체는 숨을 많이 쉬어 부족한 산소량을 보충하거나 혈액을 더 빨리 순환시키고 폐와 뇌 혈관을 확장시켜 더 많은 혈액이 흐르도록 한다. 고지대에 가면 몸은 이런 반응을 수분 후부터 수주까지 지속한다. 하지만 이런 적응력은 사람마다 달라 같은 곳에서도 사람마다 증상 차이가 있다.   고산병 사망은 대부분 고산폐부종이 원인으로 고지대에 노출된 2~3일 내에 증상이 발생한다. 초기에는 마른 기침이 나고 조금 지나면 분홍색 가래가 나오

http://www.viva100.com/main/view.php?key=20190409001652475
제목 :  [포토]한국관광공사, 일본 후쿠오카서 신한류 한국문화관광대전 열어
부제목 :  
20190409
본문 : 8일 일본 후쿠오카 JR규슈홀에서 열린 '新한류 Special Night' 에 참석한 가수 성시경이 사회자와 이야기를 나누고 있다.(사진제공=한국관광공사)한국관광공사는 지난 7일부터 9일까지 일본 후쿠오카에서 신한류, 여성, 한국여행을 테마로 한 현장체험형 "新한류 한국문화관광대전"을 개최했다고 밝혔다.임용묵 한국관광공사 후쿠오카 지사장이 7일 일본 후쿠오카 JR하카타역 광장에서 열린 '新한류 DAY'행사에서 인사말을 하고 있다.(사진제공=한국관광공사)스잔느 한국관광명예홍보대사가 7일 일본 후쿠오카 JR하카타역 광장에서 열린 '新한류 DAY'행사에서 관광토크를 하고 있다.(사진제공=한국관광공사)한국관광공사에 따르면 첫날인 7일에는 JR하카타역 광장에서 '新한류 DAY'를 개최, 한국관광명예홍보대사 스잔느의 한국관광토크쇼, 넌버벌 '판타스틱' 공연과 EDM 스페셜 스테이지, 인천/제주 등 8개 지자체의 관광홍보부스 운영, 한국음식 시식회 등의 체험행사를 열었다.둘째날 8일에는 新한류 Special Night'행사로, 가수 성시경의 스페셜콘서트와 배우 이제훈의 토크쇼를 열어 한국관광의 매력을 홍보했다.8일 일본 후쿠오카 JR규슈홀에서 열린 '新한류 Special Night' 에 참석한 가수 성시경이 열창을 하고 있다.(사진제공=한국관광공사)8일 일본 후쿠오카 JR규슈홀에서 열린 新한류 Special Night' 에 참석한 배우 이제훈이 사회자와 이야기를 나누고 있다.(사진제공=한국관광공사)마지막날 9일은 여성층을 메인타깃으로 미용, 의료, 한방, 웰니스, 드라마 등을 소개하는 '한국 여성여행 박람회'와 현지 언론과 여행업계 관계자들을 대상으로 한 '글로컬 관광설명회'도 개최했다.이철준 PD bestnews2018@viva100.com기자의 다른기

http://www.viva100.com/main/view.php?key=20190404010001557
제목 :  공주시 ‘고마열차’  6일부터 본격 운행
부제목 :  매주 토·일요일, 구석기축제 등 행사기간 운행
20190404
본문 : 공주 고마열차 운행 사진.공주시(시장 김정섭)가 오는 6일부터 대한민국 테마여행 10선 국비 공모사업의 하나로 제작·구입한 ‘고마열차’를 본격 운행한다고 밝혔다. 공주시는 그동안 3월 2주간 고마열차의 안전운행을 위해 시범운행을 하면서 차량을 점검, 보완해 왔다 공주시 마스코트인 고마곰과 공주를 형상화해 제작된 고마열차는 국토교통부의 자동차 안전기준 특례를 받은 대형 승합차량으로 동력차 1대와 객차 3대로 연결돼 있으며, 한 번에 40여명이 탑승할 수 있다.고마열차는 매주 토요일과 일요일, 공휴일 오전 10시부터 오후 5시 40분까지 1시간 간격으로 하루에 7회 운행하며 주행속도는 시속 23km 이내로 왕복 40분 정도 소요된다.주요 코스는 공산성을 출발해 송산리고분군과 공주한옥마을, 국립공주박물관을 운행하며, 운행 중에는 한국어 해설이 지원되고 좌석마다 비치된 오디오 시스템을 통해 영어와 중국어, 일본어로도 설명을 들을 수 있다.시 관계자는 “백제 역사지구 주변 관광지에 고마열차를 운행함으로써 관광객들에게 볼거리와 이동 편의를 제공할 계획”이라며, “많은 관광객과 시민들이 이색적인 고마열차에 탑승해 보길 바란다”고 말했다. 한편, 공산성에 위치한 고마열차 매표소에서는 운행 당일 오전 9시부터 오후 5시까지 이용권 구입이 가능하고 요금은 성인 3000원, 청소년 2000원, 어린이는 1000원이다.공주=강철수 기자 knews1020@naver.com기자의 다른기사보기 >
http://www.viva100.com/main/view.php?key=20190404010001424
제목 :  울주군 작천정 벚꽃... 110년 이상 된 아름드리 벚꽃 터널에 여심 '심쿵'
부제목 :  300여 그루의 벚나무 벚꽃이 장관이뤄... 상춘

http://www.viva100.com/main/view.php?key=20190402010000571
제목 :  이스타항공, 일본 휴양지 오키나와·미야자키 특가 이벤트
부제목 :  
20190402
본문 : 이스타항공 일본 휴양지 특가 이벤트 (이스타항공 제공)이스타항공이 일본 대표 휴양지 오키나와와 미야자키 노선의 특가 이벤트를 진행한다고 2일 밝혔다. ‘일본으로 떠나는 휴양, 내가 바라던 바다!’ 이벤트는 2일 오전 10시부터 오는 8일까지 일주일 간 진행되며, 판매되는 항공권의 출발 일정은 2일부터 7월31일까지다.이벤트 항공권의 편도총액운임 최저가는 △인천-오키나와 6만3300원 △인천-미야자키 6만2300원부터다.더불어 이벤트 페이지 하단에서 호텔 제휴처인 익스피디아 할인호텔의 조회 및 예약이 가능해 신속한 여행 계획이 가능하다.  이스타항공 관계자는 “오키나와는 해양 액티비티와 함께 이국적인 정취를 느낄 수 있는 곳이며, 미야자키는 온화한 기후로 골프와 함께 일본 3대 소고기 미야자키규를 맛볼 수 있는 등 다양한 먹거리와 즐길 거리가 있다”며 “이번 특가 이벤트를 통해 항공권과 호텔까지 한 번에 여행계획을 세워보시길 바란다”고 전했다.이벤트에 대한 자세한 사항은 이스타항공 홈페이지와 모바일에서 확인과 예약이 가능하다.남소라 기자 blanc@viva100.com기자의 다른기사보기 >
http://www.viva100.com/main/view.php?key=20190401010000301
제목 :  커피사며 환전까지…"보수적 금융환경 변화한다"
부제목 :  
20190401
본문 : 제1차 혁신금융심사위서 발언하는 최종구 금융위원장(사진=연합) 일본 출장 길에 오른 A씨는 차량에 탑승한 채 주문한 음식을 받는 드라이브 스루(Drive Thru) 카페에 들러 커피와 함께 출장에서 사용할 엔화를 받았다. A씨는 앞으로 지문인식만으로 보험가입이 가능한 모바일 어플리케이션을 이용해 여행자 보험에 가입하고 떠날 예정이다.드라이브 스루 매장에서 환전하고, 지문